In [1]:
# 1. 데이터 준비하기

# 코드 사전 정의

code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

In [2]:
idx2code = {v:k for k, v in code2idx.items()}
print(idx2code)

{0: 'c4', 1: 'd4', 2: 'e4', 3: 'f4', 4: 'g4', 5: 'a4', 6: 'b4', 7: 'c8', 8: 'd8', 9: 'e8', 10: 'f8', 11: 'g8', 12: 'a8', 13: 'b8'}


In [3]:
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']
# 데이터셋 생성 함수    
import numpy as np
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)
# 2. 데이터셋 생성하기
dataset = seq2dataset(seq, window_size = 4)
print(dataset)

[[11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  8]
 [ 8  1  7  8  9]
 [ 1  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 11]
 [ 9 10 11 11  4]
 [10 11 11  4 11]
 [11 11  4 11  9]
 [11  4 11  9  9]
 [ 4 11  9  9  9]
 [11  9  9  9 10]
 [ 9  9  9 10  8]
 [ 9  9 10  8  1]
 [ 9 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]
 [11  9  9  2  8]
 [ 9  9  2  8  8]
 [ 9  2  8  8  8]
 [ 2  8  8  8  8]
 [ 8  8  8  8  8]
 [ 8  8  8  8  9]
 [ 8  8  8  9  3]
 [ 8  8  9  3  9]
 [ 8  9  3  9  9]
 [ 9  3  9  9  9]
 [ 3  9  9  9  9]
 [ 9  9  9  9  9]
 [ 9  9  9  9 10]
 [ 9  9  9 10  4]
 [ 9  9 10  4 11]
 [ 9 10  4 11  9]
 [10  4 11  9  2]
 [ 4 11  9  2 10]
 [11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]]


In [4]:
# 0. 사용할 패키지 불러오기
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from keras.utils import np_utils

# 랜덤시드 고정시키기
np.random.seed(5)

# 손실 이력 클래스 정의
class LossHistory(tensorflow.keras.callbacks.Callback):
    def init(self):
        self.losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

# 입력(X)과 출력(Y) 변수로 분리하기
x_train = dataset[:,0:4]
y_train = dataset[:,4]

max_idx_value = 13

# 입력값 정규화 시키기
x_train = x_train / float(max_idx_value)


x_train = np.reshape(x_train, (50, 4, 1))

# 라벨값에 대한 one-hot 인코딩 수행
y_train = np_utils.to_categorical(y_train)

one_hot_vec_size = y_train.shape[1]

print("one hot encoding vector size is ", one_hot_vec_size)

# 3. 모델 구성하기
model = Sequential()
model.add(LSTM(128, input_shape = (4, 1)))
model.add(Dense(one_hot_vec_size, activation='softmax'))

# 4. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = LossHistory() # 손실 이력 객체 생성
history.init()

# 5. 모델 학습시키기
model.fit(x_train, y_train, epochs=2000, batch_size=10, verbose=2, callbacks=[history])

Using TensorFlow backend.


one hot encoding vector size is  12
Train on 50 samples
Epoch 1/2000
50/50 - 2s - loss: 2.4666 - accuracy: 0.1200
Epoch 2/2000
50/50 - 0s - loss: 2.4079 - accuracy: 0.3400
Epoch 3/2000
50/50 - 0s - loss: 2.3468 - accuracy: 0.3400
Epoch 4/2000
50/50 - 0s - loss: 2.2820 - accuracy: 0.3400
Epoch 5/2000
50/50 - 0s - loss: 2.1967 - accuracy: 0.3400
Epoch 6/2000
50/50 - 0s - loss: 2.0967 - accuracy: 0.3400
Epoch 7/2000
50/50 - 0s - loss: 2.0290 - accuracy: 0.3400
Epoch 8/2000
50/50 - 0s - loss: 1.9919 - accuracy: 0.3400
Epoch 9/2000
50/50 - 0s - loss: 1.9695 - accuracy: 0.3400
Epoch 10/2000
50/50 - 0s - loss: 1.9492 - accuracy: 0.3400
Epoch 11/2000
50/50 - 0s - loss: 1.9396 - accuracy: 0.3400
Epoch 12/2000
50/50 - 0s - loss: 1.9269 - accuracy: 0.3400
Epoch 13/2000
50/50 - 0s - loss: 1.9231 - accuracy: 0.3400
Epoch 14/2000
50/50 - 0s - loss: 1.9171 - accuracy: 0.3400
Epoch 15/2000
50/50 - 0s - loss: 1.9117 - accuracy: 0.3400
Epoch 16/2000
50/50 - 0s - loss: 1.9106 - accuracy: 0.3400
Epoch 17/

Epoch 139/2000
50/50 - 0s - loss: 1.6299 - accuracy: 0.4400
Epoch 140/2000
50/50 - 0s - loss: 1.6366 - accuracy: 0.3800
Epoch 141/2000
50/50 - 0s - loss: 1.6259 - accuracy: 0.4200
Epoch 142/2000
50/50 - 0s - loss: 1.6289 - accuracy: 0.3800
Epoch 143/2000
50/50 - 0s - loss: 1.6239 - accuracy: 0.4200
Epoch 144/2000
50/50 - 0s - loss: 1.6248 - accuracy: 0.4000
Epoch 145/2000
50/50 - 0s - loss: 1.6032 - accuracy: 0.4200
Epoch 146/2000
50/50 - 0s - loss: 1.6207 - accuracy: 0.3800
Epoch 147/2000
50/50 - 0s - loss: 1.6230 - accuracy: 0.4000
Epoch 148/2000
50/50 - 0s - loss: 1.6037 - accuracy: 0.4400
Epoch 149/2000
50/50 - 0s - loss: 1.6052 - accuracy: 0.4400
Epoch 150/2000
50/50 - 0s - loss: 1.6064 - accuracy: 0.4400
Epoch 151/2000
50/50 - 0s - loss: 1.6203 - accuracy: 0.4400
Epoch 152/2000
50/50 - 0s - loss: 1.5971 - accuracy: 0.4400
Epoch 153/2000
50/50 - 0s - loss: 1.5983 - accuracy: 0.4200
Epoch 154/2000
50/50 - 0s - loss: 1.6038 - accuracy: 0.4200
Epoch 155/2000
50/50 - 0s - loss: 1.5899

Epoch 276/2000
50/50 - 0s - loss: 1.3433 - accuracy: 0.5200
Epoch 277/2000
50/50 - 0s - loss: 1.3312 - accuracy: 0.5800
Epoch 278/2000
50/50 - 0s - loss: 1.3446 - accuracy: 0.5400
Epoch 279/2000
50/50 - 0s - loss: 1.3258 - accuracy: 0.5400
Epoch 280/2000
50/50 - 0s - loss: 1.3305 - accuracy: 0.5600
Epoch 281/2000
50/50 - 0s - loss: 1.3343 - accuracy: 0.5600
Epoch 282/2000
50/50 - 0s - loss: 1.3177 - accuracy: 0.5400
Epoch 283/2000
50/50 - 0s - loss: 1.3184 - accuracy: 0.5400
Epoch 284/2000
50/50 - 0s - loss: 1.3230 - accuracy: 0.5200
Epoch 285/2000
50/50 - 0s - loss: 1.3056 - accuracy: 0.5600
Epoch 286/2000
50/50 - 0s - loss: 1.3258 - accuracy: 0.5400
Epoch 287/2000
50/50 - 0s - loss: 1.3138 - accuracy: 0.5600
Epoch 288/2000
50/50 - 0s - loss: 1.3100 - accuracy: 0.5600
Epoch 289/2000
50/50 - 0s - loss: 1.3161 - accuracy: 0.5600
Epoch 290/2000
50/50 - 0s - loss: 1.3005 - accuracy: 0.5400
Epoch 291/2000
50/50 - 0s - loss: 1.3191 - accuracy: 0.5400
Epoch 292/2000
50/50 - 0s - loss: 1.3025

Epoch 413/2000
50/50 - 0s - loss: 1.0793 - accuracy: 0.6000
Epoch 414/2000
50/50 - 0s - loss: 1.0654 - accuracy: 0.6000
Epoch 415/2000
50/50 - 0s - loss: 1.0731 - accuracy: 0.5800
Epoch 416/2000
50/50 - 0s - loss: 1.0612 - accuracy: 0.6000
Epoch 417/2000
50/50 - 0s - loss: 1.0657 - accuracy: 0.5800
Epoch 418/2000
50/50 - 0s - loss: 1.0751 - accuracy: 0.5600
Epoch 419/2000
50/50 - 0s - loss: 1.0576 - accuracy: 0.5800
Epoch 420/2000
50/50 - 0s - loss: 1.0898 - accuracy: 0.6000
Epoch 421/2000
50/50 - 0s - loss: 1.0909 - accuracy: 0.5800
Epoch 422/2000
50/50 - 0s - loss: 1.0818 - accuracy: 0.6000
Epoch 423/2000
50/50 - 0s - loss: 1.0542 - accuracy: 0.6000
Epoch 424/2000
50/50 - 0s - loss: 1.0809 - accuracy: 0.5800
Epoch 425/2000
50/50 - 0s - loss: 1.0446 - accuracy: 0.5600
Epoch 426/2000
50/50 - 0s - loss: 1.0607 - accuracy: 0.5800
Epoch 427/2000
50/50 - 0s - loss: 1.0429 - accuracy: 0.6200
Epoch 428/2000
50/50 - 0s - loss: 1.0527 - accuracy: 0.5800
Epoch 429/2000
50/50 - 0s - loss: 1.0317

...

Epoch 1366/2000
50/50 - 0s - loss: 0.3554 - accuracy: 0.9200
Epoch 1367/2000
50/50 - 0s - loss: 0.4096 - accuracy: 0.8400
Epoch 1368/2000
50/50 - 0s - loss: 0.4434 - accuracy: 0.8400
Epoch 1369/2000
50/50 - 0s - loss: 0.4102 - accuracy: 0.8400
Epoch 1370/2000
50/50 - 0s - loss: 0.3675 - accuracy: 0.8800
Epoch 1371/2000
50/50 - 0s - loss: 0.3787 - accuracy: 0.8800
Epoch 1372/2000
50/50 - 0s - loss: 0.3660 - accuracy: 0.8400
Epoch 1373/2000
50/50 - 0s - loss: 0.3792 - accuracy: 0.8800
Epoch 1374/2000
50/50 - 0s - loss: 0.3362 - accuracy: 0.8600
Epoch 1375/2000
50/50 - 0s - loss: 0.3708 - accuracy: 0.8600
Epoch 1376/2000
50/50 - 0s - loss: 0.3464 - accuracy: 0.8600
Epoch 1377/2000
50/50 - 0s - loss: 0.3725 - accuracy: 0.8400
Epoch 1378/2000
50/50 - 0s - loss: 0.3527 - accuracy: 0.8600
Epoch 1379/2000
50/50 - 0s - loss: 0.4151 - accuracy: 0.8400
Epoch 1380/2000
50/50 - 0s - loss: 0.3144 - accuracy: 0.9000
Epoch 1381/2000
50/50 - 0s - loss: 0.3557 - accuracy: 0.8400
Epoch 1382/2000
50/50 - 

Epoch 1501/2000
50/50 - 0s - loss: 0.3830 - accuracy: 0.8400
Epoch 1502/2000
50/50 - 0s - loss: 0.3216 - accuracy: 0.9000
Epoch 1503/2000
50/50 - 0s - loss: 0.3256 - accuracy: 0.8600
Epoch 1504/2000
50/50 - 0s - loss: 0.3179 - accuracy: 0.8400
Epoch 1505/2000
50/50 - 0s - loss: 0.3174 - accuracy: 0.8400
Epoch 1506/2000
50/50 - 0s - loss: 0.3293 - accuracy: 0.8800
Epoch 1507/2000
50/50 - 0s - loss: 0.3240 - accuracy: 0.8800
Epoch 1508/2000
50/50 - 0s - loss: 0.3660 - accuracy: 0.8800
Epoch 1509/2000
50/50 - 0s - loss: 0.3491 - accuracy: 0.8000
Epoch 1510/2000
50/50 - 0s - loss: 0.3262 - accuracy: 0.8600
Epoch 1511/2000
50/50 - 0s - loss: 0.3195 - accuracy: 0.9000
Epoch 1512/2000
50/50 - 0s - loss: 0.3369 - accuracy: 0.8800
Epoch 1513/2000
50/50 - 0s - loss: 0.3303 - accuracy: 0.8800
Epoch 1514/2000
50/50 - 0s - loss: 0.3117 - accuracy: 0.8800
Epoch 1515/2000
50/50 - 0s - loss: 0.3266 - accuracy: 0.8600
Epoch 1516/2000
50/50 - 0s - loss: 0.3236 - accuracy: 0.8400
Epoch 1517/2000
50/50 - 

Epoch 1636/2000
50/50 - 0s - loss: 0.3090 - accuracy: 0.8400
Epoch 1637/2000
50/50 - 0s - loss: 0.3179 - accuracy: 0.8400
Epoch 1638/2000
50/50 - 0s - loss: 0.3058 - accuracy: 0.8800
Epoch 1639/2000
50/50 - 0s - loss: 0.3032 - accuracy: 0.8400
Epoch 1640/2000
50/50 - 0s - loss: 0.2905 - accuracy: 0.9000
Epoch 1641/2000
50/50 - 0s - loss: 0.3095 - accuracy: 0.8800
Epoch 1642/2000
50/50 - 0s - loss: 0.3187 - accuracy: 0.8600
Epoch 1643/2000
50/50 - 0s - loss: 0.3105 - accuracy: 0.9000
Epoch 1644/2000
50/50 - 0s - loss: 0.3144 - accuracy: 0.8600
Epoch 1645/2000
50/50 - 0s - loss: 0.3230 - accuracy: 0.8600
Epoch 1646/2000
50/50 - 0s - loss: 0.3294 - accuracy: 0.8600
Epoch 1647/2000
50/50 - 0s - loss: 0.3811 - accuracy: 0.8400
Epoch 1648/2000
50/50 - 0s - loss: 0.3081 - accuracy: 0.8800
Epoch 1649/2000
50/50 - 0s - loss: 0.3128 - accuracy: 0.9000
Epoch 1650/2000
50/50 - 0s - loss: 0.3377 - accuracy: 0.8800
Epoch 1651/2000
50/50 - 0s - loss: 0.3054 - accuracy: 0.9000
Epoch 1652/2000
50/50 - 

Epoch 1771/2000
50/50 - 0s - loss: 0.3079 - accuracy: 0.8400
Epoch 1772/2000
50/50 - 0s - loss: 0.2900 - accuracy: 0.8600
Epoch 1773/2000
50/50 - 0s - loss: 0.2904 - accuracy: 0.8800
Epoch 1774/2000
50/50 - 0s - loss: 0.3039 - accuracy: 0.8200
Epoch 1775/2000
50/50 - 0s - loss: 0.2865 - accuracy: 0.8600
Epoch 1776/2000
50/50 - 0s - loss: 0.2991 - accuracy: 0.8600
Epoch 1777/2000
50/50 - 0s - loss: 0.3027 - accuracy: 0.8800
Epoch 1778/2000
50/50 - 0s - loss: 0.3061 - accuracy: 0.9000
Epoch 1779/2000
50/50 - 0s - loss: 0.2965 - accuracy: 0.8800
Epoch 1780/2000
50/50 - 0s - loss: 0.2865 - accuracy: 0.8400
Epoch 1781/2000
50/50 - 0s - loss: 0.2905 - accuracy: 0.9000
Epoch 1782/2000
50/50 - 0s - loss: 0.2845 - accuracy: 0.9000
Epoch 1783/2000
50/50 - 0s - loss: 0.2884 - accuracy: 0.8800
Epoch 1784/2000
50/50 - 0s - loss: 0.2838 - accuracy: 0.8800
Epoch 1785/2000
50/50 - 0s - loss: 0.2954 - accuracy: 0.8800
Epoch 1786/2000
50/50 - 0s - loss: 0.2933 - accuracy: 0.8800
Epoch 1787/2000
50/50 - 

Epoch 1906/2000
50/50 - 0s - loss: 0.2969 - accuracy: 0.8400
Epoch 1907/2000
50/50 - 0s - loss: 0.2887 - accuracy: 0.8800
Epoch 1908/2000
50/50 - 0s - loss: 0.2878 - accuracy: 0.8800
Epoch 1909/2000
50/50 - 0s - loss: 0.2829 - accuracy: 0.8400
Epoch 1910/2000
50/50 - 0s - loss: 0.2884 - accuracy: 0.8800
Epoch 1911/2000
50/50 - 0s - loss: 0.2837 - accuracy: 0.8400
Epoch 1912/2000
50/50 - 0s - loss: 0.2880 - accuracy: 0.8600
Epoch 1913/2000
50/50 - 0s - loss: 0.2799 - accuracy: 0.8800
Epoch 1914/2000
50/50 - 0s - loss: 0.2824 - accuracy: 0.8600
Epoch 1915/2000
50/50 - 0s - loss: 0.2772 - accuracy: 0.9000
Epoch 1916/2000
50/50 - 0s - loss: 0.2829 - accuracy: 0.8800
Epoch 1917/2000
50/50 - 0s - loss: 0.2864 - accuracy: 0.8800
Epoch 1918/2000
50/50 - 0s - loss: 0.2795 - accuracy: 0.8600
Epoch 1919/2000
50/50 - 0s - loss: 0.2816 - accuracy: 0.8800
Epoch 1920/2000
50/50 - 0s - loss: 0.2961 - accuracy: 0.8800
Epoch 1921/2000
50/50 - 0s - loss: 0.2765 - accuracy: 0.9000
Epoch 1922/2000
50/50 - 

In [6]:
# 6. 학습과정 살펴보기
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(history.losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

# 7. 모델 평가하기
scores = model.evaluate(x_train, y_train)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))



50/50 [==============================] - 1s 10ms/sample - loss: 0.2624 - accuracy: 0.8800
accuracy: 88.00%


In [9]:
# 8. 모델 사용하기

pred_count = 50 # 최대 예측 개수 정의

# 한 스텝 예측

seq_out = ['g8', 'e8', 'e4', 'f8']
pred_out = model.predict(x_train)

for i in range(pred_count):
    idx = np.argmax(pred_out[i]) # one-hot 인코딩을 인덱스 값으로 변환
    seq_out.append(idx2code[idx]) # seq_out는 최종 악보이므로 인덱스 값을 코드로 변환하여 저장
    
print("one step prediction : ", seq_out)



one step prediction :  ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'f8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'e8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'f8', 'f8', 'f8', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']


In [11]:
# 곡 전체 예측

seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in

seq_in = [code2idx[it] / float(max_idx_value) for it in seq_in] # 코드를 인덱스값으로 변환 동시에 정규화!

for i in range(pred_count):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, 4, 1)) # batch_size, feature
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)
    seq_out.append(idx2code[idx])
    seq_in.append(idx / float(max_idx_value))
    seq_in.pop(0)      # 새로운 음표가 들어왔으니 이전 첫 음표는 삭제 하여 다음 입력으로 사용!

print("full song prediction : ", seq_out)

full song prediction :  ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'f8', 'f8', 'd4', 'c8', 'd8', 'd8', 'f8', 'g8', 'd4', 'e8', 'e8', 'e8', 'd8', 'f8', 'd8', 'd4', 'e8', 'e8', 'g8', 'd8', 'd4', 'e8', 'e8', 'g8', 'd8', 'd4', 'e8', 'e8', 'g8', 'd8', 'd4']


In [ ]:
max_idx_value